In [ ]:
def all_models(start, end, patient_id):
    models = []
    for start_ in np.arange(start, end, 30):
        for end_ in np.arange(start_ + 100, end + 1, 70):
            for reg_method in [3]:
                models.append(estimate_T(start_, end_, patient_id, reg_method))
    return models

Period = namedtuple('Period', ['start', 'end'])
def all_periods(start, end):
    periods = []
    for start_ in np.arange(start, end, 100):
        end_ = start_ + 100
        period = Period(start_, end_)
        periods.append(period)
    return periods

def models_periods_table(start, end, patient_id):
    
    if os.path.exists('pickles/table'+str(patient_id)+'-'+str(start)+'-'+str(end)) == True:
        table = pickle.load(open('pickles/table'+str(patient_id)+'-'+str(start)+'-'+str(end), 'rb'))
        models = pickle.load(open('pickles/models'+str(patient_id)+'-'+str(start)+'-'+str(end), 'rb'))
        periods = pickle.load(open('pickles/periods'+str(patient_id)+'-'+str(start)+'-'+str(end), 'rb'))
        return table, models, periods
    
    models = all_models(start, end, patient_id)
    periods = all_periods(start, end)

    table = np.empty((len(periods),len(models)), dtype=object)
    print(table.shape)

    for i in range(len(models)):
        for j in range(len(periods)):
            prediction = test_T(models[i], periods[j].start, periods[j].end, patient_id)
            table[j][i] = (prediction.data_test, prediction.data_pred)
            
    with open('pickles/table'+str(patient_id)+'-'+str(start)+'-'+str(end), 'wb') as table_file:
        pickle.dump(table, table_file)
    with open('pickles/models'+str(patient_id)+"-"+str(start)+"-"+str(end), 'wb') as models_file:
        pickle.dump(models, models_file)
    with open('pickles/periods'+str(patient_id)+"-"+str(start)+"-"+str(end), 'wb') as periods_file:
        pickle.dump(periods, periods_file)
        
    return table, models, periods

def translate_table(table, models, periods, ac_limit, mean_limit):
    clean_table = np.empty((len(periods),len(models)), dtype=object)
    for i in range(len(models)):
        for j in range(len(periods)):
            (data_test, data_pred) = table[j][i]
            error = data_test - data_pred
            
            left_rank_mean = right_rank_mean = rank_mean = extreme_mean = 0
            rank_ac = 0 # autocorelation
            
            for signal in [0,1,2]:
                if np.abs(np.mean(error[signal])) <= mean_limit:
                    rank_mean = rank_mean + 1
                if np.abs(np.mean(error[signal])) <= 0.4:
                    extreme_mean = extreme_mean + 1
                if np.abs(np.mean(error[signal][0:(int)(len(error[signal])/2)])) <= 0.7:
                    left_rank_mean = left_rank_mean + 1
                if np.abs(np.mean(error[signal][(int)(len(error[signal])/2):])) <= 0.7:
                    right_rank_mean = right_rank_mean + 1
                    
                h, pV, Q, cV = lbqtest(error[signal], range(1, min(20, len(error[signal]))), alpha=ac_limit)
                if not any(h):      
                    rank_ac = rank_ac + 1

            if (rank_ac == 3 and rank_mean == 3) or extreme_mean == 3:# and left_rank_mean == 3 and right_rank_mean == 3:
                clean_table[j][i] = 1 - np.minimum(1,np.round(np.mean(np.abs(error))/100, 5))
            else:
                clean_table[j][i] = 0
    return clean_table


def clean_table(table, models, periods):
    if len(periods) == 1:
        return table
    clean_table = np.empty((len(periods),len(models)), dtype=object)
    for i in range(len(models)):
        for j in range(len(periods)):
            clean_table[j][i] = table[j][i]
            if table[j][i] > 0:
                if (j == 0 and table[1][i] == 0) or (j == (len(periods) - 1) and table[len(periods) - 2][i] == 0) or ((j!=0) and (j!=(len(periods) - 1)) and (table[j-1][i] == 0) and (table[j+1][i]==0)):
                    clean_table[j][i] = 0
    return clean_table


def popular_model(table, models, periods):
    Period = namedtuple('Period', ['start', 'end'])
    models_popularity = np.zeros(len(models))
    model_periods = []
    for i in range(len(models)):
        pre_fit = 0
        for j in range(len(periods)):
            if pre_fit == 1 and table[j][i] > 0:
                models_popularity[i] = models_popularity[i] + 1
            if table[j][i] > 0:
                models_popularity[i] = models_popularity[i] + table[j][i]
                pre_fit = 1
            else:
                pre_fit = 0

    print("models_popularity", models_popularity)
    popular_index = np.argmax(models_popularity)
    print("popular_index", popular_index)
    to_delete = []
    for i in range(len(periods)):
        if table[i][popular_index] > 0:
            if len(model_periods) == 0 or model_periods[-1].end != periods[i].start:
                model_periods.append(periods[i])
            else:
                last_period_start = model_periods[-1].start
                model_periods.pop(-1)
                model_periods.append(Period(last_period_start, periods[i].end))
            to_delete.append(i)
    for i in reversed(to_delete):
        table = np.delete(table, i, axis=0)
        periods.pop(i)
    return table, periods, models[popular_index], model_periods



Model_Periods = namedtuple('Model_Periods', ['model', 'model_periods'])
def split_to_models(start, end, patient_id):
    
    best_rank = -np.inf
    best_results = None
    ac_limit_list = [0.1, 0.05, 0.03]
    mean_limit_list = [0.9, 1.8, 2.3, 2.8]
    for ac_limit in ac_limit_list:
        for mean_limit in mean_limit_list:
            result_rank = 0.4*(ac_limit-min(ac_limit_list))/(max(ac_limit_list)-min(ac_limit_list)) + 1 - (mean_limit-min(mean_limit_list))/(max(mean_limit_list)-min(mean_limit_list))
            print("ac_limit, mean_limit = ", ac_limit, mean_limit)
            table, models, periods = models_periods_table(start, end, patient_id)
            table = translate_table(table, models, periods, ac_limit, mean_limit)
            table = clean_table(table, models, periods)

            results = []
            for i in range(3):
                table, periods, popular_model_, model_periods = popular_model(table, models, periods)
                if len(model_periods) == 0:
                    break;
                else:
                    results.append(Model_Periods(popular_model_, model_periods))
            
            #result_rank = 0.01*result_rank + ((end-start)-len(periods)*100)/len(results)
            result_rank = 0.1*result_rank + 1-len(results)/3
            result_rank = 0.25*result_rank + ((end-start)-len(periods)*100)/(end-start)
            
            if result_rank > best_rank:
                best_rank = result_rank
                best_results = results

    for result in best_results:
        for period in result.model_periods:
            print_prediction_results(test_T(result.model, period.start, period.end, patient_id))